In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pickle
import os, sys
import copy
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib import colorbar, colors, gridspec
from metpy import plots as metpy_plots
from matplotlib.colors import ListedColormap, BoundaryNorm


HOME = Path(os.getcwd()).parents[0]

sys.path.insert(1, str(HOME))

from dsrnngan.plots import plot_precip, plot_contourf
from dsrnngan.data import denormalise, DEFAULT_LATITUDE_RANGE, DEFAULT_LONGITUDE_RANGE
from dsrnngan import data
from dsrnngan.noise import NoiseGenerator
from dsrnngan.rapsd import plot_spectrum1d, rapsd
from dsrnngan.thresholded_ranks import findthresh

In [ ]:
# model_number = 230400
model_number = 160000
model_type = 'basic'

log_folders = {'basic': '/user/work/uz22147/logs/cgan/d9b8e8059631e76f/n1000_201806-201905_e50',
               'full_image': '/user/work/uz22147/logs/cgan/43ae7be47e9a182e_full_image/n1000_201806-201905_e50',
               'cropped': '/user/work/uz22147/logs/cgan/43ae7be47e9a182e/n3000_201806-201905_e10'}

log_folder = log_folders[model_type]
with open(os.path.join(log_folder, f'arrays-{model_number}.pkl'), 'rb') as ifh:
    arrays = pickle.load(ifh)
    
truth_array = arrays['truth']
samples_gen_array = arrays['samples_gen']
fcst_array = arrays['fcst_array']
ensmean_array = np.mean(arrays['samples_gen'], axis=-1)
dates = [d[0] for d in arrays['dates']]
hours = [h[0] for h in arrays['hours']]

assert len(set(list(zip(dates, hours)))) == fcst_array.shape[0], "Degenerate date/hour combinations"
(n_samples, width, height, ensemble_size) = samples_gen_array.shape


In [ ]:
from dsrnngan.utils import load_yaml_file
# Get lat/lon range from log folder
base_folder = '/'.join(log_folder.split('/')[:-1])
config = load_yaml_file(os.path.join(base_folder, 'setup_params.yaml'))

# Locations
min_latitude = config['DATA']['min_latitude']
max_latitude = config['DATA']['max_latitude']
latitude_step_size = config['DATA']['latitude_step_size']
min_longitude = config['DATA']['min_longitude']
max_longitude = config['DATA']['max_longitude']
longitude_step_size = config['DATA']['longitude_step_size']
latitude_range=np.arange(min_latitude, max_latitude, latitude_step_size)
longitude_range=np.arange(min_longitude, max_longitude, longitude_step_size)


# Quantiles
step_size = 0.001
range_dict = {0: {'start': 0.1, 'stop': 1, 'interval': 0.1, 'marker': '+', 'marker_size': 32},
              1: {'start': 1, 'stop': 10, 'interval': 1, 'marker': '+', 'marker_size': 256},
              2: {'start': 10, 'stop': 80, 'interval':10, 'marker': '+', 'marker_size': 512},
              3: {'start': 80, 'stop': 99.1, 'interval': 1, 'marker': '+', 'marker_size': 256},
              4: {'start': 99.1, 'stop': 99.91, 'interval': 0.1, 'marker': '+', 'marker_size': 128},
              5: {'start': 99.9, 'stop': 99.99, 'interval': 0.01, 'marker': '+', 'marker_size': 32 },
              6: {'start': 99.99, 'stop': 99.999, 'interval': 0.001, 'marker': '+', 'marker_size': 10}}
                  
percentiles_list= [np.arange(item['start'], item['stop'], item['interval']) for item in range_dict.values()]
percentiles=np.concatenate(percentiles_list)
quantile_locs = [np.round(item / 100.0, 6) for item in percentiles]

## Data collection

In [ ]:
import xarray as xr

imerg_train_data = []
ifs_train_data = []
ifs_dates = []
imerg_dates = []

for year in tqdm([2016, 2017]):
    for month in range(1,13):

        imerg_ds = xr.open_dataarray(f'/user/home/uz22147/repos/rainfall_data/daily_imerg_rainfall_{month}_{year}.nc')

        imerg_data = imerg_ds.sel(lat=latitude_range, method='nearest').sel(lon=longitude_range, method='nearest').values
        
        imerg_dates += [d.astype('M8[D]').astype('O') for d in imerg_ds.time.values]

        for t in range(imerg_data.shape[0]):
            
            snapshot = imerg_data[t, :, :]
            
            imerg_train_data.append(snapshot)
        
        try:
            ifs_ds = xr.open_dataarray(f'/user/home/uz22147/repos/rainfall_data/daily_ifs_rainfall_{month}_{year}.nc')

            ifs_data = ifs_ds.sel(lat=latitude_range, method='nearest').sel(lon=longitude_range, method='nearest').values

            for t in range(ifs_data.shape[0]):
                
                snapshot = ifs_data[t, :, :]
                
                ifs_train_data.append(snapshot)
            ifs_dates += [d.astype('M8[D]').astype('O') for d in ifs_ds.time.values]

        except:
            pass

imerg_train_data = np.stack(imerg_train_data, axis = 0)
ifs_train_data = np.stack(ifs_train_data, axis = 0)

# Make dates consistent
overlapping_dates = np.array(sorted(set(ifs_dates).intersection(imerg_dates)))

imerg_overlapping_date_ix = [n for n, item in enumerate(imerg_dates) if item in overlapping_dates]
ifs_overlapping_date_ix = [n for n, item in enumerate(ifs_dates) if item in overlapping_dates]

imerg_train_data = imerg_train_data[imerg_overlapping_date_ix, :, :]
ifs_train_data = ifs_train_data[ifs_overlapping_date_ix , :, :]

In [ ]:
ifs_training_quantiles = np.quantile(ifs_train_data.flatten(), quantile_locs)
imerg_training_quantiles = np.quantile(imerg_train_data.flatten(), quantile_locs)

ifs_test_quantiles = np.quantile(fcst_array.flatten(), quantile_locs)
imerg_test_quantiles = np.quantile(truth_array.flatten(), quantile_locs)

In [ ]:
plt.plot(imerg_test_quantiles, ifs_test_quantiles)
plt.plot(imerg_test_quantiles, imerg_test_quantiles, 'o--')

# plt.plot(imerg_training_quantiles, ifs_training_quantiles)
plt.plot(imerg_training_quantiles, ifs_training_quantiles, '--')

plt.plot(imerg_test_quantiles, np.interp(ifs_test_quantiles, ifs_training_quantiles, imerg_training_quantiles))

In [ ]:
from dsrnngan.benchmarks import get_quantile_areas, get_quantiles_by_area, get_quantile_mapped_forecast

month_ranges = [[1,2], [3,4,5], [6,7,8,9], [10,11,12]]
quantile_threshold = 0.999

In [ ]:
# identify best threshold and train on all the data
quantile_areas = get_quantile_areas(list(overlapping_dates), month_ranges, latitude_range, longitude_range)
quantiles_by_area = get_quantiles_by_area(quantile_areas, fcst_data=ifs_train_data, obs_data=imerg_train_data, 
                                          quantile_locs=quantile_locs)

In [ ]:
fcst_corrected = get_quantile_mapped_forecast(fcst=fcst_array, dates=dates, 
                                              hours=hours, month_ranges=month_ranges, 
                                              quantile_areas=quantile_areas, 
                                              quantiles_by_area=quantiles_by_area)
                                            #   quantile_threshold=0.99999)

In [ ]:
import pickle

with open('/user/home/uz22147/repos/downscaling-cgan/mae_vals.pkl', 'rb') as ifh:

    mae_vals = pickle.load(ifh)

In [ ]:
y = [item[1] for item in mae_vals['mae_95']]
x = [item[0] for item in mae_vals['mae_95']] 
plt.plot(x,y)

In [ ]:
from scipy.stats import gamma
(q_99pt9, q_99pt99) = np.quantile(truth_array, [0.999, 0.9999])
fitting_data = np.random.choice(truth_array.flatten(), size=int(4e6), replace=False)

# fit_alpha, fit_loc, fit_beta=gamma.fit(fitting_data)

In [ ]:
import random

# Fitting using moments; perhaps would be better using mse distance but it is super slow
def get_gamma_parameters(data):
    mean = np.mean(data)
    variance = np.var(data)
    
    shape = mean*mean/variance
    scale = variance/mean
    
    return shape, scale

def get_lognormal_parameters(data):
    
    e_x = np.mean(data)
    var_x = np.var(data)
    
    var = np.log(1 + var_x / e_x**2)
    mu = np.log( e_x * np.exp(-0.5*var))
    
    return mu, var

# force integer values to get integer sample
shape, scale = get_gamma_parameters(truth_array)
gamma_sample_data = np.random.gamma(shape=shape, scale=scale, size=truth_array.size)

lognormal_mean, lognormal_variance = get_lognormal_parameters(truth_array)
lognormal_sample_data = np.random.lognormal(mean=lognormal_mean, sigma=np.sqrt(lognormal_variance), size=truth_array.size)
# sample_data = [i for i in gamma_random_sample(mean, variance, int(4e6))]


In [ ]:
fig, ax = plt.subplots(1,1, figsize=(5,5))

ax.hist(truth_array.flatten(), bins=np.arange(0,300,4), histtype= 'step', label='original', density=False)
ax.hist(lognormal_sample_data, histtype= 'step', label='lognormal', bins=np.arange(0,300,4), density=False)
ax.hist(gamma_sample_data, histtype= 'step', label='gamma', bins=np.arange(0,300,4), density=False)


ax.vlines(q_99pt99, 0, 10**8, linestyles='--')
ax.set_yscale('log')
ax.legend()